In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

def build_pdf_qa(pdf_path: str):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    db = DocArrayInMemorySearch.from_documents(chunks, OpenAIEmbeddings())
    return RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
        retriever=db.as_retriever()
    )

qa = build_pdf_qa("docs/cs229_lectures/attention.pdf")
query = "What problem does the Transformer architecture aim to solve?"
answer = qa.run(query)
print("Q:", query)
print("A:", answer)

loader = PyPDFLoader("docs/cs229_lectures/attention.pdf")
pages = loader.load()
print(f"Loaded {len(pages)} pages from the PDF.")

Q: What problem does the Transformer architecture aim to solve?
A: The Transformer architecture aims to solve the fundamental constraint of sequential computation by relying entirely on an attention mechanism to draw global dependencies between input and output sequences. This allows for significantly more parallelization compared to traditional models based on recurrence or convolution, potentially reaching a new state of the art in sequence transduction tasks.
Loaded 15 pages from the PDF.
